In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import os
import pandas as pd
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# prompt: help me to set the HF_TOKEN environment variable and do login to hugging face

from huggingface_hub import notebook_login
notebook_login()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Entregable 1: Clasificar las entidades nombradas de las historias clinicas

In [5]:
model = AutoModelForTokenClassification.from_pretrained(
    "anvorja/breast-cancer-biomedical-ner-sp"
)
tokenizer = AutoTokenizer.from_pretrained("anvorja/breast-cancer-biomedical-ner-sp")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [6]:
ner_pipeline = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

Device set to use cuda:0


In [25]:
import os
import pandas as pd
from nltk.tokenize import sent_tokenize   # or the splitter you already use


def procesar_historias_clinicas(path):
    """
    Re-implemented so that consecutive entities of the same type (except FECHA)
    are fused into a single row in the output DataFrame.
    """
    datos = []
    archivos = [f for f in os.listdir(path) if f.endswith(".txt")]
    total_archivos = len(archivos)
    print(f"\nSe encontraron {total_archivos} archivos de historias clínicas.\n")

    for idx, archivo in enumerate(archivos, 1):
        print(f"Procesando archivo {idx}/{total_archivos}: {archivo}")
        ruta_archivo = os.path.join(path, archivo)

        with open(ruta_archivo, encoding='utf-8') as f:
            texto = f.read()

        for i, oracion in enumerate(sent_tokenize(texto)):
            entidades = ner_pipeline(oracion)

            for ent in entidades:
                datos.append({
                    "patient_id": archivo.replace(".txt", ""),
                    "sentence_id": i,
                    "sentence": oracion,
                    "NER": ent["word"],
                    "label": ent["entity_group"],
                    "start": ent["start"],
                    "end": ent["end"]
                })

    return pd.DataFrame(datos)


In [26]:
df_ner = procesar_historias_clinicas("/content/drive/MyDrive/Tareas_Analitica_Datos_Salud/Tarea_2/Notas_Cancer_Mama/")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Se encontraron 106 archivos de historias clínicas.

Procesando archivo 1/106: 120.txt
Procesando archivo 2/106: 553284.txt
Procesando archivo 3/106: 5692361.txt
Procesando archivo 4/106: 158.txt
Procesando archivo 5/106: 139.txt
Procesando archivo 6/106: 261062.txt
Procesando archivo 7/106: 2567.txt
Procesando archivo 8/106: 236.txt
Procesando archivo 9/106: 330177_1.txt
Procesando archivo 10/106: 2071255.txt
Procesando archivo 11/106: 163.txt
Procesando archivo 12/106: 153.txt
Procesando archivo 13/106: 1010408.txt
Procesando archivo 14/106: 131.txt
Procesando archivo 15/106: 125.txt
Procesando archivo 16/106: 2102.txt
Procesando archivo 17/106: 1710.txt
Procesando archivo 18/106: 814012.txt
Procesando archivo 19/106: 130.txt
Procesando archivo 20/106: 138.txt
Procesando archivo 21/106: 137.txt
Procesando archivo 22/106: 5514.txt
Procesando archivo 23/106: 37139.txt
Procesando archivo 24/106: 69.txt
Procesando archivo 25/106: 569236.txt
Procesando archivo 26/106: 3391.txt
Procesando 

In [27]:
df_ner.head(26)

,patient_id,sentence_id,sentence,NER,label,start,end
0,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,49 años,AGE,9,16
1,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,diagnosticada,OCURRENCE_EVENT,17,30
2,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,2010,DATE,34,38
3,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,cancer de mama derecha,CANCER_CONCEPT,42,64
4,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,T1cN0M0,TNM,66,73
5,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,Ki67 30 %,BIOMARKER,75,84
6,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,Her2 negativo,BIOMARKER,87,100
7,120,1,"Paciente operada el día 22/11/2010, con mastec...",operada,OCURRENCE_EVENT,9,16
8,120,1,"Paciente operada el día 22/11/2010, con mastec...",22/11/2010,DATE,24,34
9,120,1,"Paciente operada el día 22/11/2010, con mastec...",mastectomía simple,SURGERY,40,58


In [58]:
df_ner.to_csv("/content/drive/MyDrive/Tareas_Analitica_Datos_Salud/Tarea_2/entidades_extraidas_4_anvorja_model_entregable_1.csv")

# Entregable 2: Clasificar la negación o la incertidumbre de las oraciones de las historias

In [29]:
labels = ['B-NEG', 'B-NSCO', 'B-UNC', 'B-USCO', 'I-NEG', 'I-NSCO', 'I-UNC', 'I-USCO', 'O']

id2label = {k: v for k, v in enumerate(labels)}
label2id = {v: k for k, v in enumerate(labels)}

In [30]:
neg_model = "JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES"
model = AutoModelForTokenClassification.from_pretrained(
    neg_model,
    id2label=id2label,
    label2id=label2id
)
tokenizer = AutoTokenizer.from_pretrained(neg_model)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [31]:
neg_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")


Device set to use cuda:0


In [32]:
import os
import pandas as pd
from nltk.tokenize import sent_tokenize   # or the splitter you already use


def procesar_historias_clinicas_negacion(path):
    """
    Re-implemented so that consecutive entities of the same type (except FECHA)
    are fused into a single row in the output DataFrame.
    """
    datos = []
    archivos = [f for f in os.listdir(path) if f.endswith(".txt")]
    total_archivos = len(archivos)
    print(f"\nSe encontraron {total_archivos} archivos de historias clínicas.\n")

    for idx, archivo in enumerate(archivos, 1):
        print(f"Procesando archivo {idx}/{total_archivos}: {archivo}")
        ruta_archivo = os.path.join(path, archivo)

        with open(ruta_archivo, encoding='utf-8') as f:
            texto = f.read()

        for i, oracion in enumerate(sent_tokenize(texto)):
            entidades = neg_pipeline(oracion)

            for ent in entidades:
                datos.append({
                    "patient_id": archivo.replace(".txt", ""),
                    "sentence_id": i,
                    "sentence": oracion,
                    "NER": ent["word"],
                    "label": ent["entity_group"],
                    "start": ent["start"],
                    "end": ent["end"]
                })

    return pd.DataFrame(datos)

df_ner_negacion = procesar_historias_clinicas_negacion("/content/drive/MyDrive/Tareas_Analitica_Datos_Salud/Tarea_2/Notas_Cancer_Mama/")


Se encontraron 106 archivos de historias clínicas.

Procesando archivo 1/106: 120.txt
Procesando archivo 2/106: 553284.txt
Procesando archivo 3/106: 5692361.txt
Procesando archivo 4/106: 158.txt
Procesando archivo 5/106: 139.txt
Procesando archivo 6/106: 261062.txt
Procesando archivo 7/106: 2567.txt
Procesando archivo 8/106: 236.txt
Procesando archivo 9/106: 330177_1.txt
Procesando archivo 10/106: 2071255.txt
Procesando archivo 11/106: 163.txt
Procesando archivo 12/106: 153.txt
Procesando archivo 13/106: 1010408.txt
Procesando archivo 14/106: 131.txt
Procesando archivo 15/106: 125.txt
Procesando archivo 16/106: 2102.txt
Procesando archivo 17/106: 1710.txt
Procesando archivo 18/106: 814012.txt
Procesando archivo 19/106: 130.txt
Procesando archivo 20/106: 138.txt
Procesando archivo 21/106: 137.txt
Procesando archivo 22/106: 5514.txt
Procesando archivo 23/106: 37139.txt
Procesando archivo 24/106: 69.txt
Procesando archivo 25/106: 569236.txt
Procesando archivo 26/106: 3391.txt
Procesando 

In [49]:
df_ner_negacion.head(10)

,patient_id,sentence_id,sentence,NER,label,start,end
0,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,her,NSCO,87,90
1,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,##2,NSCO,90,91
2,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,ne,NEG,92,94
3,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,##gat,NEG,94,97
4,120,0,Mujer de 49 años diagnosticada en 2010 de canc...,##ivo,NSCO,97,100
5,120,2,AC x 4 entre 29/12/2010 y 09/03/2011 + Tamoxif...,suspend,NEG,65,72
6,120,2,AC x 4 entre 29/12/2010 y 09/03/2011 + Tamoxif...,##ido,NSCO,72,75
7,120,2,AC x 4 entre 29/12/2010 y 09/03/2011 + Tamoxif...,hace 5 meses,NSCO,76,88
8,120,3,No ha sido vista en Oncologia médica desde 201...,no,NEG,0,2
9,120,3,No ha sido vista en Oncologia médica desde 201...,ha sido vista en oncologia medica desde 2016,NSCO,3,47


In [55]:
import pandas as pd
import json

def build_sentence_level_df(df_tokens: pd.DataFrame) -> pd.DataFrame:
    """
    Parameters
    ----------
    df_tokens : DataFrame
        Columns required:
        ─ patient_id, sentence_id, sentence, start, end, label

    Returns
    -------
    DataFrame
        One row per (patient_id, sentence_id, sentence) with:
        • spans : str   (JSON list of merged spans)
        • sentence_classified : str  (sentence with [TAG] in-line)
    """

    rows = []
    group_cols = ['patient_id', 'sentence_id', 'sentence']

    for (pid, sid, sent), g in df_tokens.groupby(group_cols, sort=False):
        # ---- 1. take spans for *this* sentence ---------------------------
        spans = (g[['start', 'end', 'label']]
                 .sort_values('start')
                 .query("label != 'O'")          # keep, if you use 'O'
                 .to_dict('records'))

        # ---- 2. merge adjacent sub-tokens with same label ----------------
        merged = []
        for sp in spans:
            if merged and sp['label'] == merged[-1]['label'] and sp['start'] == merged[-1]['end']:
                merged[-1]['end'] = sp['end']    # extend previous span
            else:
                merged.append(sp.copy())

        # ---- 3. build annotated sentence --------------------------------
        pieces, cursor = [], 0
        for sp in merged:
            if sp['start'] < cursor:             # overlap → skip
                continue
            pieces.append(sent[cursor:sp['start']])
            pieces.append((f"({sent[sp['start']:sp['end']]})"))
            pieces.append(f"[{sp['label']}]")
            cursor = sp['end']
        pieces.append(sent[cursor:])
        sentence_classified = ''.join(pieces)

        # ---- 4. collect row ---------------------------------------------
        rows.append({
            'patient_id': pid,
            'sentence_id': sid,
            'sentence': sent,
            'spans': json.dumps(merged, ensure_ascii=False),
            'sentence_classified': sentence_classified
        })

    return pd.DataFrame(rows)

# ------------------------------------------------------------------------
# USO:
df_sentences = build_sentence_level_df(df_ner_negacion)



In [56]:
# show entire column contents (no truncation)
pd.set_option('display.max_colwidth', None)
df_sentences[['sentence_classified']].head(10)

,sentence_classified
0,"Mujer de 49 años diagnosticada en 2010 de cancer de mama derecha, T1cN0M0, Ki67 30 % , (Her2)[NSCO] (negat)[NEG](ivo)[NSCO]."
1,"AC x 4 entre 29/12/2010 y 09/03/2011 + Tamoxifeno desde 04/2011, (suspend)[NEG](ido)[NSCO] (hace 5 meses)[NSCO]."
2,(No)[NEG] (ha sido vista en Oncologia médica desde 2016)[NSCO] y (en ginecologia desde)[NSCO] 2012.
3,(No)[NEG] (sangrado)[NSCO].
4,(No)[NEG] (líquido libre)[NSCO].
5,(No)[NEG] (palpo nodulos)[NSCO].
6,(No)[NEG] (adenopatías axilares ni SCV)[NSCO].
7,(HER)[NSCO] (2)[NSCO] (ne)[NEG](gativo)[NSCO].
8,"-Náuseas secundarias a lo previo, resueltasEvolución: Buen estado general, (as)[NEG]intomática."
9,JUICIO DIAGNÓSTICO: (Car)[USCO](cino)[NSCO](ma)[USCO] (infi)[NSCO]lt(rante)[NSCO] triple (ne)[NEG](gativo)[NSCO] cT2 cN1 M0.


In [57]:
df_sentences.to_csv("/content/drive/MyDrive/Tareas_Analitica_Datos_Salud/Tarea_2/negaciones_extraidas_entregable_2.csv")

# Entregable 3: Integrar los modelos anteriores (NER, Negación) en un sript que permita procesar las historias clínicas y producir una base de datos estructurada.

In [60]:
df = pd.read_csv("/content/drive/MyDrive/Tareas_Analitica_Datos_Salud/Tarea_2/entidades_extraidas_4_anvorja_model_entregable_1.csv")

# Agregar columna de estado
df["Estado"] = None  # Por defecto

# Agrupar por patient_id para mostrar el progreso
pacientes = df["patient_id"].unique()
print(f"\n📄 Se encontraron {len(pacientes)} pacientes para análisis de negación.\n")

# Procesar por paciente y oración
for i, pid in enumerate(pacientes, 1):
    df_paciente = df[df["patient_id"] == pid]
    oraciones_unicas = df_paciente["sentence"].unique()

    print(f"👤 Procesando paciente {i}/{len(pacientes)}: {pid} ({len(oraciones_unicas)} oraciones)")

    for j, oracion in enumerate(oraciones_unicas, 1):
        # Mostrar progreso de oración
        print(f"   ↪ Oración {j}/{len(oraciones_unicas)}")

        #predicciones = neg_pipeline(oracion)

        # Obtener entidades en esa oración
        entidades = df[(df["patient_id"] == pid) & (df["sentence"] == oracion)]

        # Buscar si alguna entidad cae dentro del texto anotado
        for idx, fila in entidades.iterrows():
            # Pasar al modelo
            predicciones = neg_pipeline(str(fila["NER"]))

            if len(predicciones) > 1:
                tipos = [pred["entity_group"] for pred in predicciones]
                if ("NEG" in tipos) or ("NSCO" in tipos):
                    df.at[idx, "Estado"] = "NEGATIVE"
                elif ("UNC" in tipos) or ("USCO" in tipos):
                    df.at[idx, "Estado"] = "UNCERTAIN"
                continue

            if len(predicciones) == 0:
                df.at[idx, "Estado"] = "POSITIVE"
                continue

            pred = predicciones[0]

            if pred["entity_group"] in ("NEG", "NSCO"):
                df.at[idx, "Estado"] = "NEGATIVE"
            elif pred["entity_group"] in ("UNC", "USCO"):
                df.at[idx, "Estado"] = "UNCERTAIN"
            else:
                df.at[idx, "Estado"] = "POSITIVE"


📄 Se encontraron 106 pacientes para análisis de negación.

👤 Procesando paciente 1/106: 120 (12 oraciones)
   ↪ Oración 1/12
   ↪ Oración 2/12
   ↪ Oración 3/12
   ↪ Oración 4/12
   ↪ Oración 5/12
   ↪ Oración 6/12
   ↪ Oración 7/12
   ↪ Oración 8/12
   ↪ Oración 9/12
   ↪ Oración 10/12
   ↪ Oración 11/12
   ↪ Oración 12/12
👤 Procesando paciente 2/106: 553284 (5 oraciones)
   ↪ Oración 1/5
   ↪ Oración 2/5
   ↪ Oración 3/5
   ↪ Oración 4/5
   ↪ Oración 5/5
👤 Procesando paciente 3/106: 5692361 (6 oraciones)
   ↪ Oración 1/6
   ↪ Oración 2/6
   ↪ Oración 3/6
   ↪ Oración 4/6
   ↪ Oración 5/6
   ↪ Oración 6/6
👤 Procesando paciente 4/106: 158 (8 oraciones)
   ↪ Oración 1/8
   ↪ Oración 2/8
   ↪ Oración 3/8
   ↪ Oración 4/8
   ↪ Oración 5/8
   ↪ Oración 6/8
   ↪ Oración 7/8
   ↪ Oración 8/8
👤 Procesando paciente 5/106: 139 (10 oraciones)
   ↪ Oración 1/10
   ↪ Oración 2/10
   ↪ Oración 3/10
   ↪ Oración 4/10
   ↪ Oración 5/10
   ↪ Oración 6/10
   ↪ Oración 7/10
   ↪ Oración 8/10
   ↪ Oraci

In [61]:
df.head(26)

,Unnamed: 0,patient_id,sentence_id,sentence,NER,label,start,end,Estado
0,0,120,0,"Mujer de 49 años diagnosticada en 2010 de cancer de mama derecha, T1cN0M0, Ki67 30 % , Her2 negativo.",49 años,AGE,9,16,POSITIVE
1,1,120,0,"Mujer de 49 años diagnosticada en 2010 de cancer de mama derecha, T1cN0M0, Ki67 30 % , Her2 negativo.",diagnosticada,OCURRENCE_EVENT,17,30,POSITIVE
2,2,120,0,"Mujer de 49 años diagnosticada en 2010 de cancer de mama derecha, T1cN0M0, Ki67 30 % , Her2 negativo.",2010,DATE,34,38,POSITIVE
3,3,120,0,"Mujer de 49 años diagnosticada en 2010 de cancer de mama derecha, T1cN0M0, Ki67 30 % , Her2 negativo.",cancer de mama derecha,CANCER_CONCEPT,42,64,POSITIVE
4,4,120,0,"Mujer de 49 años diagnosticada en 2010 de cancer de mama derecha, T1cN0M0, Ki67 30 % , Her2 negativo.",T1cN0M0,TNM,66,73,POSITIVE
5,5,120,0,"Mujer de 49 años diagnosticada en 2010 de cancer de mama derecha, T1cN0M0, Ki67 30 % , Her2 negativo.",Ki67 30 %,BIOMARKER,75,84,POSITIVE
6,6,120,0,"Mujer de 49 años diagnosticada en 2010 de cancer de mama derecha, T1cN0M0, Ki67 30 % , Her2 negativo.",Her2 negativo,BIOMARKER,87,100,NEGATIVE
7,7,120,1,"Paciente operada el día 22/11/2010, con mastectomía simple.",operada,OCURRENCE_EVENT,9,16,POSITIVE
8,8,120,1,"Paciente operada el día 22/11/2010, con mastectomía simple.",22/11/2010,DATE,24,34,POSITIVE
9,9,120,1,"Paciente operada el día 22/11/2010, con mastectomía simple.",mastectomía simple,SURGERY,40,58,POSITIVE


In [62]:
df.to_csv("/content/drive/MyDrive/Tareas_Analitica_Datos_Salud/Tarea_2/entidades_con_estado_entregable_3.csv")